In [1]:
import numpy as np
import pandas as pd
import os

In [11]:
def clean_log_3sigma(df, col):
    mean = np.log10(df[col]).mean()
    std = np.log10(df[col]).std()
    up_thresh = mean + 3 * std
    low_thresh = mean - 3 * std
    row_ind_2 = (np.log10(df[col]) > up_thresh)
    row_ind_3 = (np.log10(df[col]) < low_thresh)
    df.loc[row_ind_2, col] = np.nan
    df.loc[row_ind_3, col] = np.nan

In [14]:
folder = r"C:\Users\user\OneDrive\Desktop\new ap mort\AP data\Amman/"
files = [file for file in os.listdir(folder) if file.endswith('.xlsx')]
amm = [pd.read_excel(folder + file) for file in files]
files = [file[2:5] for file in files]
for i in range(len(files)):
    amm[i]['Sta'] = files[i]

In [ ]:
print(len(amm[amm['PM10'].isna()]))


In [19]:
len(pd.concat(amm, ignore_index=True)[pd.concat(amm, ignore_index=True)['PM10'].isna()])

6840

In [20]:
for i in range(len(amm)):
    clean_log_3sigma(amm[i], 'PM10')

In [22]:
folder = r"C:\Users\user\OneDrive\Desktop\new ap mort\AP data\Irbid/"
files = [file for file in os.listdir(folder) if file.endswith('.xlsx')]
irb = [pd.read_excel(folder + file) for file in files]
files = [file[2:5] for file in files]
for i in range(len(files)):
    irb[i]['Sta'] = files[i]

In [23]:
for i in range(len(irb)):
    clean_log_3sigma(irb[i], 'PM10')

In [27]:
folder = r"C:\Users\user\OneDrive\Desktop\new ap mort\AP data\Zarqaa/"
files = [file for file in os.listdir(folder) if file.endswith('.xlsx')]
zar = [pd.read_excel(folder + file) for file in files]
files = [file[2:5] for file in files]
for i in range(len(files)):
    zar[i]['Sta'] = files[i]

In [28]:
for i in range(len(zar)):
    clean_log_3sigma(zar[i], 'PM10')

In [29]:
amm = pd.concat(amm, ignore_index=True)
irb = pd.concat(irb, ignore_index=True)
zar = pd.concat(zar, ignore_index=True)

In [30]:
def impute(df, col):
    for day in df['Date'].unique():
        year = pd.to_datetime(day).year
        sdf = df[df['Date'] == day]
        nandf = sdf[sdf[col].isna()]
        if not nandf.empty:
            mean_value = sdf[col].mean()
            if len(nandf) == 1:
                sta_ann_mean = df[(df['Sta'] == nandf['Sta'].values[0]) & (df['Date'].dt.year == year)][col].mean()
                rest_ann_mean = df[(df['Sta'] != nandf['Sta'].values[0]) & (df['Date'].dt.year == year)][col].mean()
                df.loc[nandf.index, col] = mean_value * (sta_ann_mean / rest_ann_mean)
            if len(nandf) > 1:
                for i in range(len(nandf)):
                    sta_ann_mean = df[(df['Sta'] == nandf['Sta'].values[i]) & (df['Date'].dt.year == year)][col].mean()
                    rest_ann_mean = df[(df['Sta'] != nandf['Sta'].values[i]) & (df['Date'].dt.year == year)][col].mean()
                    df.loc[nandf.index[i], col] = mean_value * (sta_ann_mean / rest_ann_mean)

    return df

In [31]:
amm

,Date,PM10,PM2p5,NO2,SO2,CO,O3,Sta
0,2015-01-01,124.0,NaN,41.7,28.20,2463.0,NaN,GAM
1,2015-01-02,191.0,NaN,21.4,9.66,2126.0,NaN,GAM
2,2015-01-03,76.0,NaN,27.9,11.80,2480.0,NaN,GAM
3,2015-01-04,24.4,NaN,18.7,6.08,1837.0,NaN,GAM
4,2015-01-05,37.3,NaN,19.7,10.30,2032.0,NaN,GAM
...,...,...,...,...,...,...,...,...
19393,2023-11-03,NaN,39.8,23.0,15.90,NaN,NaN,YAR
19394,2023-11-04,NaN,49.7,25.1,17.40,NaN,NaN,YAR
19395,2023-11-05,NaN,46.0,NaN,NaN,NaN,NaN,YAR
19396,2023-11-06,NaN,NaN,NaN,NaN,NaN,NaN,YAR


In [32]:
amm = amm[(amm['Date'].dt.year >=2015) & (amm['Date'].dt.year <=2019)]
irb = irb[(irb['Date'].dt.year >=2015) & (irb['Date'].dt.year <=2019)]
zar = zar[(zar['Date'].dt.year >=2015) & (zar['Date'].dt.year <=2019)]

In [33]:
print(len(amm[amm['PM10'].isna()]))
print(len(irb[irb['PM10'].isna()]))
print(len(zar[zar['PM10'].isna()]))

582
697
493


In [34]:
impute(amm, 'PM10')
impute(irb, 'PM10')
impute(zar, 'PM10')

,Date,PM10,PM2p5,NO2,SO2,CO,O3,Sta
0,2015-01-01,NaN,NaN,31.8,21.80,3362.0,NaN,HAJ
1,2015-01-02,NaN,NaN,22.0,4.90,2624.0,NaN,HAJ
2,2015-01-03,106.0,NaN,26.2,7.70,2809.0,NaN,HAJ
3,2015-01-04,46.0,NaN,21.6,4.92,2460.0,NaN,HAJ
4,2015-01-05,74.1,NaN,25.8,10.50,2623.0,NaN,HAJ
...,...,...,...,...,...,...,...,...
5054,2019-12-27,43.1,NaN,14.7,3.54,NaN,NaN,MAS
5055,2019-12-28,22.6,NaN,20.6,3.62,NaN,NaN,MAS
5056,2019-12-29,70.0,NaN,29.0,4.05,NaN,NaN,MAS
5057,2019-12-30,81.2,NaN,29.0,4.07,NaN,NaN,MAS


In [35]:
print(len(amm[amm['PM10'].isna()]))
print(len(irb[irb['PM10'].isna()]))
print(len(zar[zar['PM10'].isna()]))

204
411
146


In [36]:
print(len(amm[amm['NO2'].isna()]))
print(len(irb[irb['NO2'].isna()]))
print(len(zar[zar['NO2'].isna()]))

573
188
140


In [37]:
impute(amm, 'NO2')
impute(irb, 'NO2')
impute(zar, 'NO2')

,Date,PM10,PM2p5,NO2,SO2,CO,O3,Sta
0,2015-01-01,NaN,NaN,31.8,21.80,3362.0,NaN,HAJ
1,2015-01-02,NaN,NaN,22.0,4.90,2624.0,NaN,HAJ
2,2015-01-03,106.0,NaN,26.2,7.70,2809.0,NaN,HAJ
3,2015-01-04,46.0,NaN,21.6,4.92,2460.0,NaN,HAJ
4,2015-01-05,74.1,NaN,25.8,10.50,2623.0,NaN,HAJ
...,...,...,...,...,...,...,...,...
5054,2019-12-27,43.1,NaN,14.7,3.54,NaN,NaN,MAS
5055,2019-12-28,22.6,NaN,20.6,3.62,NaN,NaN,MAS
5056,2019-12-29,70.0,NaN,29.0,4.05,NaN,NaN,MAS
5057,2019-12-30,81.2,NaN,29.0,4.07,NaN,NaN,MAS


In [38]:
print(len(amm[amm['NO2'].isna()]))
print(len(irb[irb['NO2'].isna()]))
print(len(zar[zar['NO2'].isna()]))

192
66
62


In [39]:
print(len(amm[amm['SO2'].isna()]))
print(len(irb[irb['SO2'].isna()]))
print(len(zar[zar['SO2'].isna()]))

4195
1923
135


In [40]:
impute(amm, 'SO2')
impute(irb, 'SO2')
impute(zar, 'SO2')

,Date,PM10,PM2p5,NO2,SO2,CO,O3,Sta
0,2015-01-01,NaN,NaN,31.8,21.80,3362.0,NaN,HAJ
1,2015-01-02,NaN,NaN,22.0,4.90,2624.0,NaN,HAJ
2,2015-01-03,106.0,NaN,26.2,7.70,2809.0,NaN,HAJ
3,2015-01-04,46.0,NaN,21.6,4.92,2460.0,NaN,HAJ
4,2015-01-05,74.1,NaN,25.8,10.50,2623.0,NaN,HAJ
...,...,...,...,...,...,...,...,...
5054,2019-12-27,43.1,NaN,14.7,3.54,NaN,NaN,MAS
5055,2019-12-28,22.6,NaN,20.6,3.62,NaN,NaN,MAS
5056,2019-12-29,70.0,NaN,29.0,4.05,NaN,NaN,MAS
5057,2019-12-30,81.2,NaN,29.0,4.07,NaN,NaN,MAS


In [41]:
print(len(amm[amm['SO2'].isna()]))
print(len(irb[irb['SO2'].isna()]))
print(len(zar[zar['SO2'].isna()]))

3780
1923
62


Check missing in city average

In [42]:
amm.groupby('Date')[['PM10', 'NO2', 'SO2']].mean().isna().sum()

PM10    34
NO2     32
SO2     32
dtype: int64

In [43]:
irb.groupby('Date')[['PM10', 'NO2', 'SO2']].mean().isna().sum()

PM10    39
NO2     33
SO2     97
dtype: int64

In [44]:
zar.groupby('Date')[['PM10', 'NO2', 'SO2']].mean().isna().sum()

PM10    73
NO2     31
SO2     31
dtype: int64

In [45]:
amm.groupby('Date')[['PM10', 'NO2', 'SO2']].mean().to_excel('Amman.xlsx')
irb.groupby('Date')[['PM10', 'NO2', 'SO2']].mean().to_excel('Irbid.xlsx')
zar.groupby('Date')[['PM10', 'NO2', 'SO2']].mean().to_excel('Zarqaa.xlsx')